In [1]:
import os
os. chdir('/home/users/gmenon/workspace/songsLyricsGenerator/src')

In [75]:
import torch
from torch import nn
from pytorch_lightning import LightningModule
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model
from constants.mir_constants import TrainingArgs, WAV2VEC2_ARGS
from dataclasses import dataclass, asdict
import json
import argparse
from dataclasses import dataclass

In [76]:
from flash.audio import SpeechRecognitionData

In [77]:
print(json.dumps(asdict(WAV2VEC2_ARGS), indent = 4))

{
    "TRAIN_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/train_song_metadata_en_demucs_cleaned.csv",
    "TEST_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_song_metadata_en_demucs_cleaned.csv",
    "MODEL_BACKBONE": "facebook/wav2vec2-large-960h-lv60-self",
    "BATCH_SIZE": 1,
    "NUM_EPOCHS": 15,
    "MODEL_SAVE_PATH": "/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt",
    "FINETUNE_STRATEGY": "no_freeze_deepspeed",
    "LR_SCHEDULER": "reduce_on_plateau_schedule"
}


In [5]:
datamodule = SpeechRecognitionData.from_csv("consolidated_file_path",
                                                         "transcription_capitalized",
                                                         train_file=WAV2VEC2_ARGS.TRAIN_FILE_PATH,
                                                         test_file=WAV2VEC2_ARGS.TEST_FILE_PATH,
                                                         batch_size=WAV2VEC2_ARGS.BATCH_SIZE
                                                         )

/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
datamodule

In [7]:
import flash
import torchaudio
import pytorch_lightning as pl

class SpeechDataModule(pl.LightningDataModule):
    def __init__(self, args: WAV2VEC2_ARGS):
        super().__init__()
        self.batch_size = 2

    def prepare_data(self):
        # Download and extract the audio files and transcripts, if necessary.
        pass

    def setup(self, stage=None):
        # Create the dataset
        # datamodule = 
        self.dataset = SpeechRecognitionData.from_csv("consolidated_file_path",
                                                         "transcription_capitalized",
                                                         train_file=WAV2VEC2_ARGS.TRAIN_FILE_PATH,
                                                         test_file=WAV2VEC2_ARGS.TEST_FILE_PATH,
                                                         batch_size=2
                                                         )

        # Create the dataloader
        self.train_dataloader = self.dataset.train_dataloader
        # torch.utils.data.DataLoader(
        #     self.dataset,
        #     batch_size=self.batch_size,
        #     shuffle=True,
        #     num_workers=4,
        # )
        self.val_dataloader = self.dataset.val_dataloader
        # torch.utils.data.DataLoader(
        #     self.dataset,
        #     batch_size=self.batch_size,
        #     shuffle=False,
        #     num_workers=4,
        # )

    def train_dataloader(self):
        return self.train_dataloader

    def val_dataloader(self):
        return self.val_dataloader

In [8]:
# class DALIDataset(pl.LightningDataModule):
#     def __init__(self, batch_size: int = 4,
#                   train_path :Optional[str] = None,
#                     validation_path: Optional[str] = None,
#                       model_backbone: pl.LightningModule = None,
#                       args: TrainingArgs = WAV2VEC2_ARGS
#                       ):
        
#         super().__init__()
#         self.train_path = train_path if train_path is not None else args.TRAIN_FILE_PATH
#         self.validation_path = validation_path if validation_path is not None else args.TEST_FILE_PATH
#         self.model_backbone = model_backbone if model_backbone is not None else args.MODEL_BACKBONE

#         def prepare_data(self):
#             pass
        
#         def setup(self):
#             train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH) 
#             validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)
#             songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
#             audio_dataset = Dataset.from_dict(
#                 {"audio": list(songs_metadata["file_name"]),
#                  "transcription": list(songs_metadata["transcription"])}).cast_column("audio", Audio(sampling_rate=16_000))
#             audio_dataset["transcription"] = audio_dataset["transcription"] = re.sub(WAV2VEC2_ARGS.CHARS_TO_REMOVE_FROM_TRANSCRIPTS, '', audio_dataset["transcription"]).upper()
#             audio_dataset = audio_dataset.train_test_split(test_size=0.2, shuffle=True)
            



In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
from datasets import load_dataset,Dataset,Audio
import pandas as pd
import re
import os

class SpeechRecognitionDataset(Dataset):
    def __init__(self, dataset,args:TrainingArgs,processor):
        self.model_backbone = args.MODEL_BACKBONE
        self.dataset = dataset
        self.processor = processor
        self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(self.model_backbone)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(self.model_backbone)
        self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        
    def __getitem__(self, index):
        audio = self.dataset[index]["audio"][0]
        transcription = self.dataset[index]["transcription"][0]

        input_values = self.processor(audio["array"], sampling_rate=16_000).input_values[0]
        with self.processor.as_target_processor():
            print("Entering the label encoder")
            labels = self.processor(transcription,return_tensors = 'pt').input_ids
       # attention_mask = inputs.attention_mask.squeeze()
        return input_values, labels
    
    def __len__(self):
        return len(self.dataset)



In [10]:
class SpeechRecognitionDataModule(pl.LightningDataModule):
    def __init__(self, WAV2VEC2_ARGS: WAV2VEC2_ARGS, num_workers):
        super().__init__()
        self.batch_size = WAV2VEC2_ARGS.BATCH_SIZE
        self.num_workers = num_workers
        self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.cpu_count = 1
        print(f"CPU Count = {self.cpu_count}")
        self.padding: Union[bool, str] = True
        self.max_length: Optional[int] = None
        self.max_length_labels: Optional[int] = None
        self.pad_to_multiple_of: Optional[int] = None
        self.pad_to_multiple_of_labels: Optional[int] = None
    
    def setup(self, stage=None):
        train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH).head(10)
        validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH).head(10)
        #songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
        if stage == 'fit' or stage is None:
            print("In Stage = Fit")
            train_dataset = Dataset.from_dict(
                    {"audio": list(train_df["consolidated_file_path"]),
                    "transcription": list(train_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))

            
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))

            
            train_dataset = train_dataset.map(self.transform_huggingface_dataset, num_proc = self.cpu_count)
            train_dataset =train_dataset.remove_columns(['audio','transcription','input_length'])
            self.train_dataset = train_dataset.map(self.data_collate, num_proc = self.cpu_count)

            
            val_dataset = val_dataset.map(self.transform_huggingface_dataset, num_proc = self.cpu_count,)
            val_dataset =val_dataset.remove_columns(['audio','transcription','input_length'])
            self.val_dataset = val_dataset.map(self.data_collate, num_proc = self.cpu_count)

        

        
        if stage == 'test' or stage is None:
            print("In Stage = Test")
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
          #  val_dataset["transcription"] = val_dataset["transcription"] = re.sub(WAV2VEC2_ARGS.CHARS_TO_REMOVE_FROM_TRANSCRIPTS, '', val_dataset["transcription"]).upper()
            val_dataset = val_dataset.map(self.transform_huggingface_dataset, num_proc = self.cpu_count)
            #self.test_dataset = SpeechRecognitionDataset(val_dataset,WAV2VEC2_ARGS,self.processor)
            self.test_dataset = val_dataset
    
    def train_dataloader(self):
        print("entering train data loader")
        return DataLoader(
            self.train_dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def val_dataloader(self):
        print("entering val data loader")
        return DataLoader(
            self.val_dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def test_dataloader(self):
        print("entering test data loader")
        return DataLoader(
            self.test_dataset, 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def transform_huggingface_dataset(self, batch: Dataset) -> Dataset:
        audio = batch["audio"]
        batch["input_values"] = self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
        batch["input_length"] = len(batch["input_values"])
        with self.processor.as_target_processor():
            batch["labels"] = self.processor(batch["transcription"]).input_ids
        return batch

    def data_collate(self, batch: Dataset) :
        input_features = [{"input_values": batch["input_values"]}]
        label_features = [{"input_ids": batch["labels"]}]
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )      
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

In [68]:
data = SpeechRecognitionDataModule(WAV2VEC2_ARGS, num_workers = 0)

CPU Count = 1


In [69]:
data.setup()

In Stage = Fit


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In Stage = Test


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [70]:
next(iter(data.val_dataloader()))

entering val data loader


{'input_values': [[tensor([0.0164], dtype=torch.float64),
   tensor([0.3489], dtype=torch.float64),
   tensor([0.2281], dtype=torch.float64),
   tensor([0.2110], dtype=torch.float64),
   tensor([0.5397], dtype=torch.float64),
   tensor([0.6894], dtype=torch.float64),
   tensor([0.7342], dtype=torch.float64),
   tensor([0.8010], dtype=torch.float64),
   tensor([0.9100], dtype=torch.float64),
   tensor([0.8976], dtype=torch.float64),
   tensor([1.0166], dtype=torch.float64),
   tensor([1.1703], dtype=torch.float64),
   tensor([1.1024], dtype=torch.float64),
   tensor([0.9671], dtype=torch.float64),
   tensor([1.0147], dtype=torch.float64),
   tensor([0.9596], dtype=torch.float64),
   tensor([0.6971], dtype=torch.float64),
   tensor([0.5083], dtype=torch.float64),
   tensor([0.4844], dtype=torch.float64),
   tensor([0.4458], dtype=torch.float64),
   tensor([0.3305], dtype=torch.float64),
   tensor([0.2073], dtype=torch.float64),
   tensor([0.1144], dtype=torch.float64),
   tensor([0.0438]

In [71]:
# from typing import Union, List, Dict
# import torch

# from dataclasses import dataclass, field
# from typing import Any, Dict, List, Optional, Union


# class DataCollatorCTCWithPadding:
#     """
#     Data collator that will dynamically pad the inputs received.
#     Args:
#         processor (:class:`~transformers.Wav2Vec2Processor`)
#             The processor used for proccessing the data.
#         padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
#             Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
#             among:
#             * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
#               sequence if provided).
#             * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
#               maximum acceptable input length for the model if that argument is not provided.
#             * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
#               different lengths).
#         max_length (:obj:`int`, `optional`):
#             Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
#         max_length_labels (:obj:`int`, `optional`):
#             Maximum length of the ``labels`` returned list and optionally padding length (see above).
#         pad_to_multiple_of (:obj:`int`, `optional`):
#             If set will pad the sequence to a multiple of the provided value.
#             This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
#             7.5 (Volta).
#     """

#     processor: Wav2Vec2Processor
#     padding: Union[bool, str] = True
#     max_length: Optional[int] = None
#     max_length_labels: Optional[int] = None
#     pad_to_multiple_of: Optional[int] = None
#     pad_to_multiple_of_labels: Optional[int] = None

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         # split inputs and labels since they have to be of different lengths and need
#         # different padding methods
#         input_features = [{"input_values": feature["input_values"]} for feature in features]
#         label_features = [{"input_ids": feature["labels"]} for feature in features]

#         batch = self.processor.pad(
#             input_features,
#             padding=self.padding,
#             max_length=self.max_length,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors="pt",
#         )
#         with self.processor.as_target_processor():
#             labels_batch = self.processor.pad(
#                 label_features,
#                 padding=self.padding,
#                 max_length=self.max_length_labels,
#                 pad_to_multiple_of=self.pad_to_multiple_of_labels,
#                 return_tensors="pt",
#             )

#         # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         batch["labels"] = labels

#         return batch

In [78]:
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
from datasets import load_dataset,Dataset,Audio
import pandas as pd
import re
import os

class SpeechRecognitionDataModule(pl.LightningDataModule):
    def __init__(self, WAV2VEC2_ARGS: WAV2VEC2_ARGS, num_workers):
        super().__init__()
        self.batch_size = WAV2VEC2_ARGS.BATCH_SIZE
        self.num_workers = num_workers
        self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.cpu_count = 1
        print(f"CPU Count = {self.cpu_count}")
    
    def setup(self, stage=None):
        train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH)
        validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)
        #songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
        if stage == 'fit' or stage is None:
            print("In Stage = Fit")
            train_dataset = Dataset.from_dict(
                    {"audio": list(train_df["consolidated_file_path"]),
                    "transcription": list(train_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.train_dataset = train_dataset.map(self.prepare_dataset,remove_columns = train_dataset.column_names)
            
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.val_dataset = val_dataset.map(self.prepare_dataset,remove_columns = val_dataset.column_names)
            #common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)

        
        if stage == 'test' or stage is None:
            print("In Stage = Test")
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))

            self.test_dataset = val_dataset
    
    def train_dataloader(self):
        print("entering train data loader")
        return DataLoader(
            self.train_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def val_dataloader(self):
        print("entering val data loader")
        return DataLoader(
            self.val_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
    
    def test_dataloader(self):
        print("entering test data loader")
        return DataLoader(
            self.test_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers
        )
        
    def prepare_dataset(self, batch):
        audio = batch["audio"]
        transcription = batch["transcription"]
        batch["input_values"] = audio["array"]
        batch["labels"] = self.tokenizer(transcription, padding=True, truncation=True).input_ids
        batch["input_length"] = len(batch["input_values"])
        batch["target_length"] = len(batch["labels"])
        return batch
    #tokenizer(batch["target_text"], **additional_kwargs).input_ids

In [106]:
import torch
from torch import nn
from pytorch_lightning import LightningModule, Trainer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC

class Wav2SeqModel(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.batch_size = hparams.batch_size
        # self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(hparams.wav2vec2_model)
        # self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(hparams.wav2vec2_model)
        # self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.wav2vec2 = AutoModelForCTC.from_pretrained(hparams.wav2vec2_model,ctc_zero_infinity=True,ctc_loss_reduction="mean")
        self.wav2vec2.freeze_feature_encoder()
        # print(self.wav2vec2.config)
        self.seq2seq = AutoModelForCausalLM.from_pretrained(hparams.lm_model)
        self.enc_to_dec_proj = nn.Linear(self.wav2vec2.config.hidden_size, self.seq2seq.config.hidden_size)
        print(self.seq2seq.config)
        #self.data_collator = DataCollatorCTCWithPadding()


    def forward(self, audio):
        #print("entering forward step")
        x = self.wav2vec2(audio[0],output_hidden_states=True,output_attentions=True)
        predicted_ids = torch.argmax(x.logits, dim=-1)
        predicted_ids = predicted_ids[:,:512]
        print(predicted_ids.shape)
        #x_ids = self.enc_to_dec_proj(predicted_ids)
        #x_hidden_states = self.enc_to_dec_proj(x.hidden_states[0])
        #print(f"hidden states = {x.hidden_states[0].shape},{x.hidden_states[0]}")
        # print(f"attentions = {len(x.attentions[0])}, {x.attentions}")
        # print(f"logits = {len(x.logits[0])},{x.logits}")
        logits = self.seq2seq(input_ids = predicted_ids).logits #,attention_mask = x.attentions, decoder_input_ids=predicted_ids

            #     decoder_outputs = self.decoder(
            # input_ids=decoder_input_ids,
            # attention_mask=decoder_attention_mask,
            # encoder_hidden_states=encoder_hidden_states,
            # encoder_attention_mask=encoder_attention_mask,
            # inputs_embeds=decoder_inputs_embeds,
            # output_attentions=output_attentions,
            # output_hidden_states=output_hidden_states,
            # use_cache=use_cache,
            # past_key_values=past_key_values,
            # return_dict=return_dict,
        return logits

    def training_step(self, batch, batch_idx):
        #print("entering training step")
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        labels = labels.reshape(-1)
        # input_length = batch["input_length"]
        # target_length = batch["input_length"]
        #audio,labels,attention_mask = batch
        # audio = batch["input_values"]
        # labels = batch["labels"]
        # print(audio)
        # print(labels)
        logits = self(audio)
        logits = logits.reshape(-1, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch,batch_idx):
        #print("entering validation step")
       # print(batch)
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        labels = labels.reshape(-1)
        # input_length = batch["input_length"]
        # target_length = batch["input_length"]
        #audio,labels,attention_mask = batch
        # audio = batch["input_values"]
        # labels = batch["labels"]
        # print(audio)
        # print(labels)
        logits = self(audio)
        logits = logits.reshape(-1, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        print(logits.shape,labels.shape,input_lengths.shape,target_lengths.shape)
        #loss = nn.CTCLoss(blank=0).forward(logits, labels,input_length,target_length)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        #loss_fct = nn.CrossEntropyLoss()
        #loss = loss_fct(logits.reshape(-1, self.seq2seq.config.vocab_size), labels.reshape(-1))
        self.log('val_loss', loss, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=hparams.learning_rate)

# {'input': tensor([[-0.0004, -0.0008, -0.0004,  ..., -0.0038, -0.0031,  0.0000]],
#        device='cuda:0'), 'target': ['SOMETHING IS NOT THE SAME'], <DataKeys.METADATA: 'metadata'>: [{'sampling_rate': 16000}]}


    # def validation_step(self, batch, batch_idx):
    #     x, y = batch
    #     y_hat = self.model(x)
    #     loss = F.cross_entropy(y_hat, y)
    #     self.log("val_loss", loss)


In [108]:

import pytorch_lightning as pl
hparams = argparse.Namespace()
hparams.wav2vec2_model = 'facebook/wav2vec2-base'
hparams.lm_model = 'bert-base-uncased' #'facebook/bart-large'
hparams.vocab_size = 10000
hparams.learning_rate = 0.001
hparams.batch_size = WAV2VEC2_ARGS.BATCH_SIZE

model = Wav2SeqModel(hparams)
trainer = Trainer(max_epochs=1,devices=1, accelerator="gpu")
trainer.fit(model,SpeechRecognitionDataModule(WAV2VEC2_ARGS,num_workers=4))

# {'audio': [{'path': None, 'array': array([-0.05114746, -0.11273193, -0.09152222, ...,  0.2796936 ,
#         0.29998779,  0.20550537]), 'sampling_rate': 16000}, {'path': None, 'array': array([ 0.14523315,  0.05508423, -0.13534546, ...,  0.01959229,
#         0.00973511, -0.00811768]), 'sampling_rate': 16000}], 'transcription': ["just a tryin' to survive", "don't go on me"], 'input_values':

/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

CPU Count = 1
In Stage = Fit


Map:   0%|          | 0/4342 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1086 [00:00<?, ? examples/s]

...

Training: 0it [00:00, ?it/s]

torch.Size([1, 75])
torch.Size([1, 152])
torch.Size([1, 249])
torch.Size([1, 87])
torch.Size([1, 374])
torch.Size([1, 86])
torch.Size([1, 55])
torch.Size([1, 292])
torch.Size([1, 60])
torch.Size([1, 168])
torch.Size([1, 70])
torch.Size([1, 298])
torch.Size([1, 99])
torch.Size([1, 102])
torch.Size([1, 236])
torch.Size([1, 45])
torch.Size([1, 88])
torch.Size([1, 98])
torch.Size([1, 112])
torch.Size([1, 147])
torch.Size([1, 104])
torch.Size([1, 90])
torch.Size([1, 109])
torch.Size([1, 231])
torch.Size([1, 191])
torch.Size([1, 107])
torch.Size([1, 252])
torch.Size([1, 152])
torch.Size([1, 127])
torch.Size([1, 150])
torch.Size([1, 60])
torch.Size([1, 169])
torch.Size([1, 95])
torch.Size([1, 97])
torch.Size([1, 49])
torch.Size([1, 137])
torch.Size([1, 47])
torch.Size([1, 124])
torch.Size([1, 124])
torch.Size([1, 43])
torch.Size([1, 49])
torch.Size([1, 272])
torch.Size([1, 34])
torch.Size([1, 65])
torch.Size([1, 142])
torch.Size([1, 45])
torch.Size([1, 180])
torch.Size([1, 114])
torch.Size([1

Validation: 0it [00:00, ?it/s]

torch.Size([1, 143])
torch.Size([143, 30522]) torch.Size([28]) torch.Size([1]) torch.Size([1])
torch.Size([1, 165])
torch.Size([165, 30522]) torch.Size([22]) torch.Size([1]) torch.Size([1])
torch.Size([1, 153])
torch.Size([153, 30522]) torch.Size([26]) torch.Size([1]) torch.Size([1])
torch.Size([1, 344])
torch.Size([344, 30522]) torch.Size([31]) torch.Size([1]) torch.Size([1])
torch.Size([1, 86])
torch.Size([86, 30522]) torch.Size([19]) torch.Size([1]) torch.Size([1])
torch.Size([1, 60])
torch.Size([60, 30522]) torch.Size([9]) torch.Size([1]) torch.Size([1])
torch.Size([1, 74])
torch.Size([74, 30522]) torch.Size([17]) torch.Size([1]) torch.Size([1])
torch.Size([1, 222])
torch.Size([222, 30522]) torch.Size([30]) torch.Size([1]) torch.Size([1])
torch.Size([1, 82])
torch.Size([82, 30522]) torch.Size([26]) torch.Size([1]) torch.Size([1])
torch.Size([1, 115])
torch.Size([115, 30522]) torch.Size([25]) torch.Size([1]) torch.Size([1])
torch.Size([1, 72])
torch.Size([72, 30522]) torch.Size([26]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [109]:
import librosa 
def create_audio_tensor(audio_path):
    audio, sample_rate = librosa.load(audio_path)
    audio_tensor = torch.from_numpy(audio).float()
    audio_tensor = audio_tensor.unsqueeze(0)
    return audio_tensor
audio_tensor = create_audio_tensor("notebooks/separated/mdx_extra/test_clip/vocals.wav")
print(audio_tensor.unsqueeze(0).shape)
model.eval()
with torch.no_grad():
    y_hat = model(audio_tensor.unsqueeze(0))
torch.argmax(y_hat, dim=-1)

torch.Size([1, 1, 47149])
torch.Size([1, 147])


tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]])

In [110]:
y_hat

tensor([[[  0.0388, -10.8294, -10.5336,  ..., -19.0243, -21.0487, -15.3293],
         [  0.0388, -10.8294, -10.5336,  ..., -19.0243, -21.0487, -15.3293],
         [  0.0388, -10.8294, -10.5336,  ..., -19.0243, -21.0487, -15.3293],
         ...,
         [  0.0388, -10.8294, -10.5336,  ..., -19.0243, -21.0487, -15.3293],
         [  0.0388, -10.8294, -10.5336,  ..., -19.0243, -21.0487, -15.3293],
         [  0.0388, -10.8294, -10.5336,  ..., -19.0243, -21.0487, -15.3293]]])

In [70]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC

def create_audio_tensor(audio_path):
    audio, sample_rate = librosa.load(audio_path)
    audio_tensor = torch.from_numpy(audio).float()
    audio_tensor = audio_tensor.unsqueeze(0)
    return audio_tensor

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
audio_tensor = create_audio_tensor("notebooks/separated/mdx_extra/test_clip/vocals.wav")
logits = model(audio_tensor).logits
predictions = torch.argmax(logits, dim=-1)
print(audio_tensor)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[-0.1152, -0.1827, -0.1892,  ..., -0.2025, -0.2312, -0.1570]])


In [71]:
predictions

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  0,  0,  9,  0,  0, 21,  0,  0,
          0,  0,  0,  5, 13,  0,  0,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0, 13,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  5,
         13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,
          0,  0,  0]])

In [93]:
import torch
from torch import nn
from pytorch_lightning import LightningModule, Trainer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC

class Wav2SeqModel(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.batch_size = hparams.batch_size
        # self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(hparams.wav2vec2_model)
        # self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(hparams.wav2vec2_model)
        # self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.wav2vec2 = AutoModelForCTC.from_pretrained(hparams.wav2vec2_model,ctc_zero_infinity=True,ctc_loss_reduction="mean")
        self.wav2vec2.freeze_feature_encoder()
        # print(self.wav2vec2.config)
        self.seq2seq = AutoModelForCausalLM.from_pretrained(hparams.lm_model)
        self.enc_to_dec_proj = nn.Linear(self.wav2vec2.config.hidden_size, self.seq2seq.config.hidden_size)
        print(self.seq2seq.config)
        #self.data_collator = DataCollatorCTCWithPadding()


    def forward(self, audio):
        print("entering forward step")
        x = self.wav2vec2(audio[0],output_hidden_states=True,output_attentions=True)
        predicted_ids = torch.argmax(x.logits, dim=-1)
        #x_ids = self.enc_to_dec_proj(predicted_ids)
        #x_hidden_states = self.enc_to_dec_proj(x.hidden_states[0])
        #print(f"hidden states = {x.hidden_states[0].shape},{x.hidden_states[0]}")
        # print(f"attentions = {len(x.attentions[0])}, {x.attentions}")
        # print(f"logits = {len(x.logits[0])},{x.logits}")
        logits = self.seq2seq(input_ids = predicted_ids).logits #,attention_mask = x.attentions, decoder_input_ids=predicted_ids

            #     decoder_outputs = self.decoder(
            # input_ids=decoder_input_ids,
            # attention_mask=decoder_attention_mask,
            # encoder_hidden_states=encoder_hidden_states,
            # encoder_attention_mask=encoder_attention_mask,
            # inputs_embeds=decoder_inputs_embeds,
            # output_attentions=output_attentions,
            # output_hidden_states=output_hidden_states,
            # use_cache=use_cache,
            # past_key_values=past_key_values,
            # return_dict=return_dict,
        return logits

    def training_step(self, batch, batch_idx):
        print("entering training step")
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        labels = labels.reshape(-1)
        # input_length = batch["input_length"]
        # target_length = batch["input_length"]
        #audio,labels,attention_mask = batch
        # audio = batch["input_values"]
        # labels = batch["labels"]
        # print(audio)
        # print(labels)
        logits = self(audio)
        logits = logits.reshape(-1, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch,batch_idx):
        print("entering validation step")
        print(batch)
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        labels = labels.reshape(-1)
        # input_length = batch["input_length"]
        # target_length = batch["input_length"]
        #audio,labels,attention_mask = batch
        # audio = batch["input_values"]
        # labels = batch["labels"]
        # print(audio)
        # print(labels)
        logits = self(audio)
        logits = logits.reshape(-1, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        print(logits.shape,labels.shape,input_lengths.shape,target_lengths.shape)
        #loss = nn.CTCLoss(blank=0).forward(logits, labels,input_length,target_length)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        #loss_fct = nn.CrossEntropyLoss()
        #loss = loss_fct(logits.reshape(-1, self.seq2seq.config.vocab_size), labels.reshape(-1))
        self.log('val_loss', loss, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=hparams.learning_rate)

# {'input': tensor([[-0.0004, -0.0008, -0.0004,  ..., -0.0038, -0.0031,  0.0000]],
#        device='cuda:0'), 'target': ['SOMETHING IS NOT THE SAME'], <DataKeys.METADATA: 'metadata'>: [{'sampling_rate': 16000}]}


    # def validation_step(self, batch, batch_idx):
    #     x, y = batch
    #     y_hat = self.model(x)
    #     loss = F.cross_entropy(y_hat, y)
    #     self.log("val_loss", loss)


In [94]:
train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH)
validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)

In [95]:
train_df[train_df.transcription_capitalized.str.len()>200]

,Folder,File,consolidated_file_path,Length,file_name,transcription,language,transcription_capitalized
